<a href="https://colab.research.google.com/github/CharmStrange/PySpark/blob/main/proto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://granulate.io/blog/5-pyspark-optimization-techniques-you-should-know/#Use-DataFrame-Dataset-over-RDD

https://sparkbyexamples.com/pyspark-rdd/#what-is-rdd

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=eacee7bc61de2ad6b54710f931c274e9cb6256c02592613573b2d01bee9bbd29
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [17]:
import requests
import re
from bs4 import BeautifulSoup
import sqlite3
from pyspark.sql import SparkSession

regions = {
    "서울특별시": "https://www.daangn.com/region/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C",
    "부산광역시": "https://www.daangn.com/region/%EB%B6%80%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C",
    "대구광역시": "https://www.daangn.com/region/%EB%8C%80%EA%B5%AC%EA%B4%91%EC%97%AD%EC%8B%9C",
    "인천광역시": "https://www.daangn.com/region/%EC%9D%B8%EC%B2%9C%EA%B4%91%EC%97%AD%EC%8B%9C",
    "광주광역시": "https://www.daangn.com/region/%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C",
    "대전광역시": "https://www.daangn.com/region/%EB%8C%80%EC%A0%84%EA%B4%91%EC%97%AD%EC%8B%9C",
    "울산광역시": "https://www.daangn.com/region/%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C",
    "세종특별자치시": "https://www.daangn.com/region/%EC%84%B8%EC%A2%85%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EC%8B%9C",
    "경기도": "https://www.daangn.com/region/%EA%B2%BD%EA%B8%B0%EB%8F%84",
    "강원특별자치도": "https://www.daangn.com/region/%EA%B0%95%EC%9B%90%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EB%8F%84",
    "충청북도": "https://www.daangn.com/region/%EC%B6%A9%EC%B2%AD%EB%B6%81%EB%8F%84",
    "충청남도": "https://www.daangn.com/region/%EC%B6%A9%EC%B2%AD%EB%82%A8%EB%8F%84",
    "전라북도": "https://www.daangn.com/region/%EC%A0%84%EB%9D%BC%EB%B6%81%EB%8F%84",
    "전라남도": "https://www.daangn.com/region/%EC%A0%84%EB%9D%BC%EB%82%A8%EB%8F%84",
    "경상북도": "https://www.daangn.com/region/%EA%B2%BD%EC%83%81%EB%B6%81%EB%8F%84",
    "경상남도": "https://www.daangn.com/region/%EA%B2%BD%EC%83%81%EB%82%A8%EB%8F%84",
    "제주특별자치도": "https://www.daangn.com/region/%EC%A0%9C%EC%A3%BC%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EB%8F%84"
}

spark = SparkSession.builder.appName("proto").config("spark.some.config.option", "some-value").getOrCreate()

titles = []
prices = []
for region, url in regions.items():
  response = requests.get(url)
  html_content = response.text
  soup = BeautifulSoup(html_content, "html.parser")
  card_title_elements = soup.find_all(class_="card-title")
  card_price_elements = soup.find_all(class_="card-price")

  for element in card_title_elements:
    titles.append(element.get_text(strip=True))

  for element in card_price_elements:
    text = element.get_text(strip=True)
    # 숫자가 아닌 문자는 모두 제거하고, '만'을 뒤에 붙입니다.
    text = re.sub(r'[^0-9]', '', text) + '만'
    # '만'이 포함된 경우에만 값을 처리합니다.
    if '만' in text and text != '만':
        # '만'을 숫자로 치환하여 곱해줍니다.
        tmp = int(text.replace('만', '')) * 10000
        prices.append(tmp)


rdd_titles = spark.sparkContext.parallelize(titles)
rdd_prices = spark.sparkContext.parallelize(prices)


In [18]:
print(rdd_titles.count())
print(rdd_prices.count())

print(rdd_titles.first())
print(rdd_prices.first())

#rdd_titles.saveAsTextFile("rdd_titles")
#rdd_prices.saveAsTextFile("rdd_prices")

1679
1655
s22울트라 512G
2000000000


In [ ]:
def create_table():
    conn = sqlite3.connect('CarrotText_WHOLE.db')
    c = conn.cursor()

    c.execute('''CREATE TABLE IF NOT EXISTS CarrotTextWHOLE
                 (Title TEXT, Price INTEGER)''')

    conn.commit()
    conn.close()

def insert_data(CarrotTextWHOLE):
    conn = sqlite3.connect('CarrotText_WHOLE.db')
    c = conn.cursor()

    try:
        c.execute('''INSERT INTO CarrotText_WHOLE VALUES (?, ?)''',
                  (CarrotTextWHOLE.get('Title:', ''), CarrotTextWHOLE.get('Price:', '')))
    except KeyError as e:
        print(e)
    else:
        conn.commit()
    finally:
        conn.close()